In [14]:
import argparse
import numpy as np
import random
import signal
from MLUtils import get_MJPolicyGradient
import Player, Game
import MoveGenerator
from test_cases import utils
%load_ext autoreload
%autoreload 2
import MoveGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
EXIT_FLAG = False
names = ["Amy", "Billy", "Clark", "David"]
freq_shuffle_players = 8
freq_model_save = None
game_record_size = 100
game_record_count = 0

game_record = np.zeros((game_record_size, 4,2))
pg_model_dir = "/home/ubuntu/telebot/test2"
trainer_models = {
    "heuristics": {
        "class": MoveGenerator.RuleBasedAINaive,
        "parameters":{
             "display_step": False,
             "s_chow": 2,
             "s_pong": 6,
             "s_future": 1,
             "s_explore": 0,
             "s_neighbor_suit": 0,
             "s_mixed_suit": 0
        }
    },
    "policy_gradient": {
        "class": MoveGenerator.PGGenerator,
        "parameters":{
            "display_step": False,
            "pg_model_path": pg_model_dir,
            "is_train": False,
            "skip_history": False
        }
    },
    "random": {
        "class": MoveGenerator.RandomGenerator,
        "parameters":{
            "display_step": False
        }
    }
}

In [48]:
pg_model_paras = {
	"is_deep": True,
	"learning_rate": 1e-3,
	"reward_decay": 0.99
}
model_dir = pg_model_dir
action = "train"

In [55]:
players = []
for i in range(3):
    player = Player.Player(MoveGenerator.RandomGenerator, player_name=names[i],**trainer_models["random"]["parameters"])
    players.append(player)                                                                                                            
'''
model = get_MJPolicyGradient(model_dir, **pg_model_paras)    
pg_player = Player.Player(MoveGenerator.PGGenerator, player_name = names[3], pg_model_path = model_dir, skip_history = False, is_train = action == "train", display_step = action == "play")
'''
pg_player = Player.Player(MoveGenerator.MyPGGenerator,player_name = names[3], pg_model_path = model_dir, skip_history = False, is_train = action == "train", display_step = False)
players.append(pg_player)

pg_model = pg_player.move_generator.model
Ma
#player = Player.Player(MoveGenerator.Human,player_name=names[3])
#players.append(player)

Load Model
pass reading
New Network
finished building graph


In [56]:
players

[Amy, Billy, Clark, David]

In [57]:
game = Game.Game(players)

In [58]:
winner, losers, penalty = game.start_game()

In [ ]:
for i in range(10000000):
    game = Game.Game(players)
    winner, losers, penalty = game.start_game()
    
    #print(i)
    
    if action == "train":
        pg_model.learn(display_cost = (i+1) % game_record_size == 0)
    
    index = game_record_count%game_record_size
    game_record_count += 1
    
    game_record[index, :, 0] = 0
    game_record[index, :, 1] = 0
    
    
    
    if winner is not None:
        winner_id = players.index(winner)
        game_record[index, winner_id, 0] = 1
        game_record[index, winner_id, 1] = 0
        for loser in losers:
            loser_id = players.index(loser)
            game_record[index, loser_id, 1] = 1
            game_record[index, loser_id, 0] = 0
    if (i+1) % game_record_size == 0:
        print("#%5d: %.2f%%/%.2f%%\t%.2f%%/%.2f%%\t%.2f%%/%.2f%%\t%.2f%%/%.2f%%"%(i+1, game_record[:, 0, 0].mean()* 100, game_record[:, 0, 1].mean()* 100,
                                                                                            game_record[:, 1, 0].mean()* 100, game_record[:, 1, 1].mean()* 100, 
                                                                                            game_record[:, 2, 0].mean()* 100, game_record[:, 2, 1].mean()* 100, 
                                                                                            game_record[:, 3, 0].mean()* 100, game_record[:, 3, 1].mean()* 100))

#8604: -10927.2842, invalide: 0.4900
#  100: 1.00%/3.00%	2.00%/3.00%	1.00%/0.00%	1.00%/1.00%
#8704: -9414.6113, invalide: 0.4898
#  200: 0.00%/3.00%	0.00%/3.00%	3.00%/0.00%	0.00%/3.00%
#8804: -8367.0967, invalide: 0.4897
#  300: 0.00%/0.00%	0.00%/0.00%	0.00%/0.00%	0.00%/0.00%
#8904: -10409.5049, invalide: 0.4898
#  400: 0.00%/1.00%	1.00%/1.00%	0.00%/1.00%	1.00%/1.00%
#9004: -11233.0332, invalide: 0.4906
#  500: 0.00%/1.00%	0.00%/3.00%	2.00%/0.00%	1.00%/1.00%
#9104: -10268.2783, invalide: 0.4910
#  600: 2.00%/1.00%	0.00%/3.00%	3.00%/1.00%	1.00%/3.00%
#9204: -12518.2363, invalide: 0.4900
#  700: 2.00%/1.00%	1.00%/0.00%	1.00%/1.00%	0.00%/2.00%
#9304: -12381.9697, invalide: 0.4890
#  800: 0.00%/0.00%	0.00%/0.00%	0.00%/1.00%	1.00%/0.00%
#9404: -11998.7764, invalide: 0.4898
#  900: 2.00%/3.00%	2.00%/2.00%	1.00%/1.00%	0.00%/1.00%
#9504: -10948.6924, invalide: 0.4893
# 1000: 2.00%/1.00%	0.00%/2.00%	0.00%/1.00%	1.00%/1.00%
#9604: -11723.1250, invalide: 0.4893
# 1100: 1.00%/2.00%	0.00%/2.00%	2.0